# Python Expert system based on CLIPS

## installation 

In [1]:
! pip install experta

## The Basics

An expert system is a program capable of pairing up a set of **facts** with
a set of **rules** to those facts, and execute some actions based on the matching rules.

## Facts

`Facts` are the basic unit of information of Experta. They are used by the system to reason about the problem.

Let's enumerate some facts about `Facts`, so... metafacts ;)



The class `Fact` is a subclass of `dict`.


In [2]:
from experta import *
f = Fact(a=1, b=2)
f['a']

1

In contrast to `dict`, you can create a `Fact` without keys (only values), and `Fact` will create a numeric index for your values.

In [3]:
f = Fact('x', 'y', 'z') 
f[0]

'x'

You can mix autonumeric values with key-values, but autonumeric must be declared first

In [0]:
f = Fact('x', 'y', 'z', a=1, b=2)

In [5]:
f[1] # using index 

'y'

In [6]:
f['b']

2

## Facts 

You can subclass `Fact` to express different kinds of data or extend it with your custom functionality.

In [0]:
class Alert(Fact):
    """The alert level."""
    pass

class Status(Fact):
    """The system status."""
    pass

f1 = Alert('red')
f2 = Status('critical')

## Rules


In Experta a **rule** is a callable, decorated with `Rule`.

Rules have two components, LHS (left-hand-side) and RHS
(right-hand-side).

* The *LHS* describes (using **patterns**) the conditions on which the rule * should be executed (or fired).

* The *RHS* is the set of actions to perform when the rule is fired.

For a `Fact` to match a `Pattern`, all pattern restrictions must be
**True** when the `Fact` is evaluated against it.


In [0]:
class MyFact(Fact):
  pass

@Rule(MyFact())  # This is the LHS
def match_with_every_myfact():
  """This rule will match with every instance of `MyFact`."""
  # This is the RHS
  pass

@Rule(Fact('animal', family='felinae'))
def match_with_cats():
  """
  Match with every `Fact` which:

    * f[0] == 'animal'
    * f['family'] == 'felinae'

  """
  print("Meow!") 

## DefFacts

Most of the time expert systems needs a set of facts to be present for the system to work. This is the purpose of the DefFacts decorator.

All DefFacts inside a KnowledgeEngine will be called every time the reset method is called.





In [0]:
@DefFacts()
def needed_data():
    yield Fact(best_color="red")
    yield Fact(best_body="medium")
    yield Fact(best_sweetness="dry")

## KnowledgeEngine

This is the place where all the magic happens.

The first step is to make a subclass of it and use Rule to decorate its methods.

After that, you can instantiate it, populate it with facts, and finally run it.

In [10]:
engine = KnowledgeEngine()
engine.reset()
engine.declare(Fact(Color='Blue'))

Fact(Color='Blue')

In [11]:
engine.facts

FactList([(0, InitialFact()), (1, Fact(Color='Blue'))])

### retract
Removes an existing fact from the factlist.

### modify
Retracts some fact from the factlist and declares a new one with some changes. Changes are passed as arguments.

In [12]:
engine.facts

FactList([(0, InitialFact()), (1, Fact(Color='Blue'))])

In [13]:
engine.modify(engine.facts[1], color='yellow', blink=True)

Fact(Color='Blue', color='yellow', blink=True)

In [14]:
engine.facts

FactList([(0, InitialFact()),
          (2, Fact(Color='Blue', color='yellow', blink=True))])

## A simple Example about trafic light 

In [15]:
from random import choice
from experta import *


class Light(Fact):
  """Info about the traffic light."""
  pass


class RobotCrossStreet(KnowledgeEngine):
  @Rule(Light(color='green'))
  def green_light(self):
      print("Walk")

  @Rule(Light(color='red'))
  def red_light(self):
      print("Don't walk")

  @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
  def cautious(self, light):
      print("Be cautious because light is", light["color"])

engine = RobotCrossStreet()
engine.reset()
color_now = choice(['green', 'yellow', 'blinking-yellow', 'red'])
print('color now:', color_now)
engine.declare(Light(color=color_now))
engine.run()

color now: blinking-yellow
Be cautious because light is blinking-yellow


## Animal Example 

In [16]:
# (defrule duck animal-is duck) => (assert (sound-is quack))) 

class Animal(Fact):
  """Info about the traffic light."""
  pass


class AnimalKE1(KnowledgeEngine):
  @Rule(Animal('cat'))
  def cat_sound(self):
      print("mew")

  @Rule(Animal('duck'))
  def duck_sound(self):
      print("quack")


engine = AnimalKE1()
engine.reset()
animal_type = choice(['duck', 'cat', 'horse'])
print('animal:', animal_type)
engine.declare(Animal(animal_type))
engine.run()

animal: cat
mew


In [17]:
# (defrule is-it-a-duck
#   (animal-has webbed-feet)
#   (animal-has feathers)
#   =>
#   (assert (animal-is duck)))

class Animal(Fact):
  """Info about the traffic light."""
  pass


class AnimalKE2(KnowledgeEngine):
  @Rule(AND( Animal(has=('feathers')), Animal(has=('feathers'))))
  def guess_animal(self):
    print("Animal is duck")
    print("Sound is quack")


engine = AnimalKE2()
engine.reset()
engine.declare(Animal(has='feathers'), Animal(has='webbed-feet'))
engine.run()

Animal is duck
Sound is quack


## conclude a fact 

In [18]:
class Animal(Fact):
  """Info about the traffic light."""
  pass


class AnimalKE3(KnowledgeEngine):
  @Rule(AND( Animal(has=('feathers')), Animal(has=('feathers'))))
  def guess_animal(self):
    self.declare(Animal('duck'))
    print(self.facts)
  
  @Rule(Animal('duck'))
  def it_is_duck(self):
    print("Animal is duck")
    print("Sound is quack")

engine = AnimalKE3()
engine.reset()
engine.declare(Animal(has='feathers'), Animal(has='webbed-feet'))
engine.run()
engine.run()

<f-0>: InitialFact()
<f-1>: Animal(has='feathers')
<f-2>: Animal(has='webbed-feet')
<f-3>: Animal('duck')
Animal is duck
Sound is quack
